# Extract Response Activations from GCG Jailbreaks

This notebook extracts activations during **response generation** (not prompt encoding) to capture the model's semantic state when:
1. **Complying** with harmful requests (GCG jailbreak successful)
2. **Refusing** harmful requests (same prompt, no GCG suffix)

**Key Insight:** GCG suffixes work via attention hijacking at the prompt level. But the response activations capture the model's genuine semantic state - whether it's in "compliance mode" or "refusal mode".

**Outputs:**
- `response_activations_compliant.pt` - Activations during harmful response generation
- `response_activations_refusing.pt` - Activations during refusal generation
- `response_activations_labeled.pt` - Combined dataset with labels

**Requirements:** A100 GPU (40GB) recommended

## Section 1: Setup & Installation

In [ ]:
# Colab setup - clone repo and install dependencies
import os
import sys

# Check if running in Colab
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    # Clone repo if not already cloned
    if not os.path.exists('JB_mech'):
        !git clone https://github.com/ChuloIva/JB_mech.git
    
    %cd JB_mech
    
    # Install dependencies
    !pip install -q transformers==4.47.0 pandas pyarrow datasets einops omegaconf safetensors accelerate
    !pip install -q baukit@git+https://github.com/davidbau/baukit.git
    
    # Install GLP package
    !pip install -q -e third_party/generative_latent_prior
    
    # Add to path explicitly
    sys.path.insert(0, 'third_party/generative_latent_prior')
    
    print("Colab setup complete")
else:
    # Local setup
    os.chdir('/Users/ivanculo/Desktop/Projects/JB_mech')
    print("Local setup complete")

In [ ]:
# Core imports
import gc
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from typing import List, Dict, Tuple, Optional

# Add src to path and import project modules
sys.path.insert(0, "src")
sys.path.insert(0, "third_party/generative_latent_prior")

from jb_mech.config import (
    MODEL_NAME, TARGET_LAYER, HIDDEN_SIZE,
    add_third_party_to_path, ensure_output_dirs
)

add_third_party_to_path()
ensure_output_dirs()

# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Configuration
N_SAMPLES = 200          # Number of jailbreak examples to process
LAYER = 15               # Layer to extract activations from (GLP trained on layer 15)
MAX_NEW_TOKENS = 100     # Tokens to generate for each response
RANDOM_SEED = 42

# Output paths
OUTPUT_DIR = Path("outputs/response_activations")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Layer: {LAYER}")
print(f"  Samples: {N_SAMPLES}")
print(f"  Max new tokens: {MAX_NEW_TOKENS}")
print(f"  Output: {OUTPUT_DIR}")

## Section 2: Load Data

In [ ]:
# Load the successful jailbreaks CSV
jailbreaks_path = "data/gcg-evaluated-data/llama-3.1-8b-instruct_successful_jailbreaks.csv"
df = pd.read_csv(jailbreaks_path)

print(f"Total successful jailbreaks: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")

# Sample N examples
df_sample = df.sample(n=min(N_SAMPLES, len(df)), random_state=RANDOM_SEED)
print(f"\nSampled {len(df_sample)} examples")

# Extract prompts
prompts_clean = df_sample['message_str'].tolist()           # Harmful prompt only (will refuse)
prompts_jailbreak = df_sample['message_suffixed'].tolist()  # Harmful prompt + GCG suffix (will comply)

# Preview
print(f"\n--- Example ---")
print(f"Clean prompt: {prompts_clean[0][:100]}...")
print(f"Jailbreak prompt: {prompts_jailbreak[0][:150]}...")

## Section 3: Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()
print(f"Model loaded on {model.device}")

## Section 4: Response Activation Extraction

We extract activations **during response generation**, not during prompt encoding. This captures the model's semantic state while it's actively generating compliant/refusing text.

In [ ]:
def format_chat_prompt(prompt: str, tokenizer) -> str:
    """Format prompt using chat template."""
    messages = [{"role": "user", "content": prompt}]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


def extract_response_activations(
    model,
    tokenizer,
    prompts: List[str],
    layer: int,
    max_new_tokens: int = 100,
    extraction_points: List[str] = ["first", "middle", "last"],
    batch_size: int = 1,
) -> Dict[str, torch.Tensor]:
    """
    Extract activations during response generation.
    
    Args:
        model: HuggingFace model
        tokenizer: HuggingFace tokenizer
        prompts: List of prompts to generate responses for
        layer: Layer to extract activations from
        max_new_tokens: Max tokens to generate
        extraction_points: Which response tokens to extract ["first", "middle", "last", "mean"]
        batch_size: Batch size for generation (1 recommended for memory)
    
    Returns:
        Dict with keys for each extraction point, values are (n_prompts, hidden_dim) tensors
    """
    results = {point: [] for point in extraction_points}
    responses = []
    
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating responses"):
        batch_prompts = prompts[i:i+batch_size]
        
        # Format prompts
        formatted = [format_chat_prompt(p, tokenizer) for p in batch_prompts]
        
        # Tokenize
        inputs = tokenizer(
            formatted,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        ).to(model.device)
        
        prompt_length = inputs['input_ids'].shape[1]
        
        # Storage for activations during generation
        response_activations = []
        
        def hook(module, input, output):
            # output[0] is the hidden state: (batch, seq, hidden)
            # We only want the last token's activation (the one just generated)
            h = output[0][:, -1, :].detach().cpu()
            response_activations.append(h)
        
        # Register hook
        handle = model.model.layers[layer].register_forward_hook(hook)
        
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,  # Greedy for reproducibility
                pad_token_id=tokenizer.eos_token_id,
                return_dict_in_generate=True,
            )
        
        handle.remove()
        
        # Stack response activations: (n_generated_tokens, batch, hidden)
        if len(response_activations) > 0:
            response_acts = torch.stack(response_activations, dim=0)
            n_tokens = response_acts.shape[0]
            
            # Extract at specified points
            for point in extraction_points:
                if point == "first" and n_tokens > 0:
                    results[point].append(response_acts[0])  # First generated token
                elif point == "middle" and n_tokens > 0:
                    mid_idx = n_tokens // 2
                    results[point].append(response_acts[mid_idx])
                elif point == "last" and n_tokens > 0:
                    results[point].append(response_acts[-1])  # Last generated token
                elif point == "mean" and n_tokens > 0:
                    results[point].append(response_acts.mean(dim=0))  # Mean over all tokens
        
        # Decode response for inspection
        generated_ids = outputs.sequences[:, prompt_length:]
        response_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        responses.extend(response_text)
        
        # Cleanup
        del inputs, outputs, response_activations
        torch.cuda.empty_cache()
    
    # Stack results
    for point in extraction_points:
        if len(results[point]) > 0:
            results[point] = torch.cat(results[point], dim=0)  # (n_prompts, hidden)
    
    return results, responses

print("Response activation extraction function defined")

## Section 5: Extract Compliant Activations (GCG Jailbreaks)

In [ ]:
print("="*80)
print("EXTRACTING COMPLIANT ACTIVATIONS (Jailbreak prompts)")
print("="*80)
print(f"Processing {len(prompts_jailbreak)} jailbreak prompts...")
print("These should produce harmful responses (model complying)\n")

compliant_activations, compliant_responses = extract_response_activations(
    model,
    tokenizer,
    prompts_jailbreak,
    layer=LAYER,
    max_new_tokens=MAX_NEW_TOKENS,
    extraction_points=["first", "middle", "last", "mean"],
)

print(f"\nCompliant activations extracted:")
for point, acts in compliant_activations.items():
    print(f"  {point}: {acts.shape}")

# Preview some responses
print(f"\n--- Sample Compliant Responses ---")
for i in range(min(3, len(compliant_responses))):
    print(f"\n[{i+1}] {compliant_responses[i][:200]}...")

## Section 6: Extract Refusing Activations (Clean Prompts)

In [ ]:
print("="*80)
print("EXTRACTING REFUSING ACTIVATIONS (Clean prompts, no GCG)")
print("="*80)
print(f"Processing {len(prompts_clean)} clean prompts...")
print("These should produce refusal responses\n")

refusing_activations, refusing_responses = extract_response_activations(
    model,
    tokenizer,
    prompts_clean,
    layer=LAYER,
    max_new_tokens=MAX_NEW_TOKENS,
    extraction_points=["first", "middle", "last", "mean"],
)

print(f"\nRefusing activations extracted:")
for point, acts in refusing_activations.items():
    print(f"  {point}: {acts.shape}")

# Preview some responses
print(f"\n--- Sample Refusing Responses ---")
for i in range(min(3, len(refusing_responses))):
    print(f"\n[{i+1}] {refusing_responses[i][:200]}...")

## Section 7: Verify Response Categories

Check that compliant responses are actually harmful and refusing responses are actually refusals.

In [ ]:
def classify_response(response: str) -> str:
    """Simple heuristic to classify response as compliant or refusing."""
    response_lower = response.lower()
    
    refusal_phrases = [
        "i cannot", "i can't", "i'm not able", "i am not able",
        "i won't", "i will not", "i'm unable", "i am unable",
        "i apologize", "sorry", "i'm sorry",
        "cannot assist", "can't help", "cannot help",
        "against my", "not appropriate", "not able to",
        "decline", "refuse", "harmful", "dangerous",
        "illegal", "unethical", "violates",
    ]
    
    for phrase in refusal_phrases:
        if phrase in response_lower[:200]:  # Check beginning of response
            return "refusing"
    
    return "compliant"


# Classify all responses
compliant_classifications = [classify_response(r) for r in compliant_responses]
refusing_classifications = [classify_response(r) for r in refusing_responses]

# Statistics
compliant_accuracy = sum(1 for c in compliant_classifications if c == "compliant") / len(compliant_classifications)
refusing_accuracy = sum(1 for c in refusing_classifications if c == "refusing") / len(refusing_classifications)

print(f"Response Classification Results:")
print(f"  Jailbreak prompts → Compliant responses: {compliant_accuracy*100:.1f}%")
print(f"  Clean prompts → Refusing responses: {refusing_accuracy*100:.1f}%")

# Filter to only keep correctly classified examples
valid_compliant_mask = [c == "compliant" for c in compliant_classifications]
valid_refusing_mask = [c == "refusing" for c in refusing_classifications]

print(f"\nValid samples:")
print(f"  Compliant: {sum(valid_compliant_mask)} / {len(valid_compliant_mask)}")
print(f"  Refusing: {sum(valid_refusing_mask)} / {len(valid_refusing_mask)}")

In [ ]:
# Filter activations to only valid examples
def filter_activations(activations_dict, mask):
    """Filter activations by boolean mask."""
    mask_tensor = torch.tensor(mask)
    filtered = {}
    for point, acts in activations_dict.items():
        filtered[point] = acts[mask_tensor]
    return filtered

compliant_activations_filtered = filter_activations(compliant_activations, valid_compliant_mask)
refusing_activations_filtered = filter_activations(refusing_activations, valid_refusing_mask)

print(f"Filtered activations:")
print(f"  Compliant: {compliant_activations_filtered['mean'].shape}")
print(f"  Refusing: {refusing_activations_filtered['mean'].shape}")

## Section 8: Analyze Activation Differences

In [ ]:
# Compute mean activations for each category
compliant_mean = compliant_activations_filtered['mean'].mean(dim=0)  # (hidden_dim,)
refusing_mean = refusing_activations_filtered['mean'].mean(dim=0)    # (hidden_dim,)

# Compute "compliance direction" = compliant - refusing
compliance_direction = compliant_mean - refusing_mean
compliance_direction_normalized = F.normalize(compliance_direction, dim=0)

print(f"Compliance direction computed:")
print(f"  Shape: {compliance_direction.shape}")
print(f"  Norm: {compliance_direction.norm().item():.4f}")

# Cosine similarity between compliant and refusing means
cosine_sim = F.cosine_similarity(compliant_mean.unsqueeze(0), refusing_mean.unsqueeze(0)).item()
print(f"  Cosine similarity (compliant vs refusing): {cosine_sim:.4f}")

In [ ]:
# Project all activations onto compliance direction
compliant_projs = (compliant_activations_filtered['mean'] @ compliance_direction_normalized).numpy()
refusing_projs = (refusing_activations_filtered['mean'] @ compliance_direction_normalized).numpy()

# Visualize separation
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(refusing_projs, bins=30, alpha=0.6, label='Refusing', color='blue', density=True)
plt.hist(compliant_projs, bins=30, alpha=0.6, label='Compliant', color='red', density=True)
plt.xlabel('Projection onto Compliance Direction')
plt.ylabel('Density')
plt.title('Separation of Response Activations')
plt.legend()
plt.axvline(0, color='black', linestyle='--', alpha=0.5)

plt.subplot(1, 2, 2)
data = [refusing_projs, compliant_projs]
bp = plt.boxplot(data, labels=['Refusing', 'Compliant'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
plt.ylabel('Projection onto Compliance Direction')
plt.title('Response Activation Distributions')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "response_activation_separation.png", dpi=150)
plt.show()

# Statistics
print(f"\nSeparation Statistics:")
print(f"  Refusing mean projection: {refusing_projs.mean():.4f} (std: {refusing_projs.std():.4f})")
print(f"  Compliant mean projection: {compliant_projs.mean():.4f} (std: {compliant_projs.std():.4f})")
print(f"  Separation: {compliant_projs.mean() - refusing_projs.mean():.4f}")

# Compute classification accuracy with simple threshold
threshold = (compliant_projs.mean() + refusing_projs.mean()) / 2
compliant_correct = (compliant_projs > threshold).sum() / len(compliant_projs)
refusing_correct = (refusing_projs < threshold).sum() / len(refusing_projs)
print(f"\nLinear separability (threshold={threshold:.4f}):")
print(f"  Compliant correctly classified: {compliant_correct*100:.1f}%")
print(f"  Refusing correctly classified: {refusing_correct*100:.1f}%")
print(f"  Overall accuracy: {(compliant_correct + refusing_correct) / 2 * 100:.1f}%")

## Section 9: Compare with Prompt-Level Jailbreak Direction

Load the prompt-level jailbreak direction from notebook 02 and compare with response-level compliance direction.

In [ ]:
# Try to load the prompt-level jailbreak direction
jailbreak_dir_path = Path("outputs/jailbreak_direction_analysis.pt")

if jailbreak_dir_path.exists():
    jailbreak_results = torch.load(jailbreak_dir_path)
    jailbreak_dir = jailbreak_results['jailbreak_dir_normalized']
    
    # Compare directions
    cosine_prompt_response = F.cosine_similarity(
        jailbreak_dir.unsqueeze(0),
        compliance_direction_normalized.unsqueeze(0)
    ).item()
    
    print(f"Comparison: Prompt-level vs Response-level directions")
    print(f"  Cosine similarity: {cosine_prompt_response:.4f}")
    
    if abs(cosine_prompt_response) > 0.5:
        print(f"  → Directions are {'aligned' if cosine_prompt_response > 0 else 'anti-aligned'}")
        print(f"  → This suggests prompt and response capture similar/opposite phenomena")
    else:
        print(f"  → Directions are largely orthogonal")
        print(f"  → This suggests prompt (attention) and response (semantic) are different")
else:
    print(f"Prompt-level jailbreak direction not found at {jailbreak_dir_path}")
    print("Run notebook 02 first to generate it.")

## Section 10: Train Simple Safety Probe

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Prepare data
X_compliant = compliant_activations_filtered['mean'].numpy()
X_refusing = refusing_activations_filtered['mean'].numpy()

X = np.vstack([X_refusing, X_compliant])
y = np.array([0] * len(X_refusing) + [1] * len(X_compliant))  # 0=refusing, 1=compliant

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y)

print(f"Training data: {X_train.shape}")
print(f"Test data: {X_test.shape}")
print(f"Class balance - Train: {y_train.mean():.2f}, Test: {y_test.mean():.2f}")

In [ ]:
# Train logistic regression probe
probe = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
probe.fit(X_train, y_train)

# Evaluate
y_pred = probe.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Safety Probe Results:")
print(f"  Test Accuracy: {accuracy*100:.1f}%")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Refusing', 'Compliant']))

# Extract probe direction (the weights)
probe_direction = torch.tensor(probe.coef_[0], dtype=torch.float32)
probe_direction_normalized = F.normalize(probe_direction, dim=0)

# Compare with our simple compliance direction
cosine_probe_compliance = F.cosine_similarity(
    probe_direction_normalized.unsqueeze(0),
    compliance_direction_normalized.unsqueeze(0)
).item()
print(f"\nProbe direction vs simple compliance direction:")
print(f"  Cosine similarity: {cosine_probe_compliance:.4f}")

## Section 11: Save Results

In [ ]:
# Save all results
results = {
    # Activations
    "compliant_activations": compliant_activations_filtered,
    "refusing_activations": refusing_activations_filtered,
    
    # Directions
    "compliance_direction": compliance_direction,
    "compliance_direction_normalized": compliance_direction_normalized,
    "probe_direction": probe_direction,
    "probe_direction_normalized": probe_direction_normalized,
    
    # Responses (for inspection)
    "compliant_responses": [r for r, v in zip(compliant_responses, valid_compliant_mask) if v],
    "refusing_responses": [r for r, v in zip(refusing_responses, valid_refusing_mask) if v],
    
    # Metadata
    "layer": LAYER,
    "n_compliant": len(compliant_activations_filtered['mean']),
    "n_refusing": len(refusing_activations_filtered['mean']),
    "probe_accuracy": accuracy,
    
    # Prompts
    "prompts_jailbreak": [p for p, v in zip(prompts_jailbreak, valid_compliant_mask) if v],
    "prompts_clean": [p for p, v in zip(prompts_clean, valid_refusing_mask) if v],
}

# Save main results
output_path = OUTPUT_DIR / "response_activations_labeled.pt"
torch.save(results, output_path)
print(f"Results saved to: {output_path}")

# Save just the compliance direction for easy loading
compliance_dir_path = OUTPUT_DIR / "compliance_direction.pt"
torch.save({
    "direction": compliance_direction,
    "direction_normalized": compliance_direction_normalized,
    "probe_direction": probe_direction,
    "probe_direction_normalized": probe_direction_normalized,
    "layer": LAYER,
}, compliance_dir_path)
print(f"Compliance direction saved to: {compliance_dir_path}")

In [ ]:
# Summary
print("\n" + "="*80)
print("SUMMARY: Response Activation Extraction")
print("="*80)

print(f"\n1. Data Extracted:")
print(f"   Compliant (jailbreak) responses: {len(compliant_activations_filtered['mean'])}")
print(f"   Refusing (clean) responses: {len(refusing_activations_filtered['mean'])}")

print(f"\n2. Compliance Direction:")
print(f"   This direction points from 'refusing' to 'compliant' state")
print(f"   Norm: {compliance_direction.norm().item():.4f}")

print(f"\n3. Linear Separability:")
print(f"   Safety probe accuracy: {accuracy*100:.1f}%")
print(f"   → Response activations ARE linearly separable")

print(f"\n4. Key Insight:")
print(f"   Unlike prompt-level 'jailbreak direction' (attention artifact),")
print(f"   the response-level 'compliance direction' captures genuine")
print(f"   semantic state: is the model complying or refusing?")

print(f"\n5. Next Steps:")
print(f"   - Use compliance direction with GLP for manifold analysis")
print(f"   - Query Activation Oracle about compliant vs refusing states")
print(f"   - Test if this direction transfers across models")

print(f"\nOutputs saved to: {OUTPUT_DIR}")

In [ ]:
# Cleanup
del model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Cleanup complete")